<h3>Imports</h3>

In [29]:
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pandas as pd

<h3>Defining Paths</h3>

In [30]:
train_path = 'content/dataset/cnn/pneumonia_revamped/train/'
test_path = 'content/dataset/cnn/pneumonia_revamped/test/'
valid_path = 'content/dataset/cnn/pneumonia_revamped/val/'

<h3>Setting Image Size</h3>

In [31]:
img_height, img_width = 224, 224

<h3>Data Augmentation</h3>

In [32]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [33]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=16,
    class_mode='binary',
    color_mode='grayscale' 
)
validation_generator = test_datagen.flow_from_directory(
    valid_path,
    target_size=(img_height, img_width),
    batch_size=16,
    class_mode='binary',
    color_mode='grayscale'
)

Found 4192 images belonging to 2 classes.
Found 1040 images belonging to 2 classes.


<h3>Function to Preprocess greyscale images into RGB</h3>

<h3>Load VGG16</h3>

In [34]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 1))  # Updated for grayscale

ValueError: The input must have 3 channels; Received `input_shape=(224, 224, 1)`

<h3>Freeze base model layers for fine-tuning</h3>

In [23]:
for layer in base_model.layers:
    layer.trainable = False

<h3>Add custom layers for classification</h3>

In [24]:
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Regularization to prevent overfitting
predictions = Dense(1, activation='sigmoid')(x)

<h3>Create the final model</h3>

In [25]:
model = Model(inputs=base_model.input, outputs=predictions)

<h3>Compile Model</h3>

In [26]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

<h3>Early Stopping</h3>

In [27]:
early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=3)
learning_rate_reduction = ReduceLROnPlateau(monitor="val_loss", patience=2, verbose=1, factor=0.3, min_lr=0.000001)
callbacks_list = [early_stopping, learning_rate_reduction]

<h3>Model Training</h3>

In [28]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator),
    callbacks=callbacks_list
)

Epoch 1/20


2024-04-04 10:52:55.768067: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-04-04 10:52:56.512879: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at conv_ops_fused_impl.h:760 : INVALID_ARGUMENT: input depth must be evenly divisible by filter depth: 1 vs 3
2024-04-04 10:52:56.512912: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: input depth must be evenly divisible by filter depth: 1 vs 3
	 [[{{node model_1/block1_conv1/Relu}}]]


InvalidArgumentError: Graph execution error:

Detected at node 'model_1/block1_conv1/Relu' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/vibhav/lung/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "/home/vibhav/lung/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/home/vibhav/lung/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start
      self.io_loop.start()
    File "/home/vibhav/lung/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
      self._run_once()
    File 